In [1]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys
import os

from dotenv import load_dotenv, find_dotenv

ROOT = Path('../')
DATA_DIR = ROOT/'tests'/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

import countdowner as cd

load_dotenv(find_dotenv())


True

In [2]:
# Read a watchlist

w = cd.read_watchlist(DATA_DIR/'watchlist.yaml')
w


{'name': 'test',
 'email_addresses': ['brainbummer@mailinator.com', 'rhymedude@mailinator.com'],
 'products':     description       stock_code
 0   Brazil nuts           291156
 1  GB chocolate            32467
 2          fake  fake_stock_code
 3        cheese           281739}

In [4]:
# Get and parse a product

code = w['products']['stock_code'].iat[0]
r = cd.get_product(32467)
cd.parse_product(r)

{'stock_code': '32467',
 'name': 'Green & Blacks Chocolate Block Dark 85% Cocoa',
 'description': "Passionately crafted by bringing together the finest ingredients, green & black's offers a distinctively delicious, smooth chocolate experience. Green symbolises our commitment to ethically sourced cocoa, & black stands for the high quality and delicious taste of our chocolate, for pure chocolate pleasure. Green & black's organic 85% dark chocolate is made with a generous measure of madagascan vanilla to balance the bitter notes of the cocoa, creating chocolate that is intense and smooth.",
 'size': '90g',
 'unit_price_($)': 3.33,
 'unit_size': '100G',
 'sale_price_($)': 3.0,
 'price_($)': 3.8,
 'discount_(%)': 21.1,
 'datetime': '2020-06-19T15:19:42'}

In [5]:
# Get and parse many products

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes)
f

CPU times: user 80.8 ms, sys: 9.05 ms, total: 89.9 ms
Wall time: 617 ms


,stock_code,name,description,size,unit_price_($),unit_size,sale_price_($),price_($),discount_(%),datetime
3,281739,Mainland Cheese Block Organic Cheddar,Mainland organic cheddar is a mild cheddar che...,500g,20.00,1KG,10.0,16.0,37.5,2020-06-19 15:19:48
1,32467,Green & Blacks Chocolate Block Dark 85% Cocoa,Passionately crafted by bringing together the ...,90g,3.33,100G,3.0,3.8,21.1,2020-06-19 15:19:47
0,291156,Macro Brazil Nuts,None,250g,2.60,100G,6.5,6.5,0.0,2020-06-19 15:19:47
2,fake_stock_code,None,None,None,NaN,None,NaN,NaN,NaN,2020-06-19 15:19:48


In [8]:
# Filter sales

cd.filter_sales(f)

,name,price_($),discount_(%)
0,Mainland Cheese Block Organic Cheddar,16.0,37.5
1,Green & Blacks Chocolate Block Dark 85% Cocoa,3.8,21.1


In [ ]:
# Email sales
print(f"Emailing to {w['email_addresses']}")

username = os.getenv('GMAIL_USERNAME')  # Replace with your GMail username
password = os.getenv('GMAIL_PASSWORD')  # Replace with your GMail password
cd.email(f, w['email_addresses'], "Hello from Countdowner", username, password)

In [ ]:
# Get products in bulk

cd.run_pipeline(DATA_DIR/'watchlist.yaml')

In [ ]:
# Run pipeline again: read watchlist, collect products, write products to file, 
# filter sales, and email results

cd.run_pipeline(DATA_DIR/'watchlist.yaml', out_path=OUT_DIR/'products.csv')
%less {OUT_DIR/'products.csv'}